### Baselines:
- BERT-based classifier trained on the data

### Ideas:
- Using NLI model: https://huggingface.co/microsoft/deberta-base-mnli and fine-tuning it on the data we have.
- Validity on the premise (sentence) level.

In [1]:
import json
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

In [2]:
from datasets import load_dataset, load_metric, Dataset, Split
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline, DebertaForSequenceClassification
from transformers import TrainingArguments, Trainer
import wandb
import torch
from tqdm import tqdm

In [3]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'recall' : recall,
        'precision': precision,
        'f1': f1,
    }

In [4]:
nli_tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base-mnli')
nli_model     = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base-mnli')
arg_stance_pipeline = TextClassificationPipeline(model=nli_model, tokenizer=nli_tokenizer, framework='pt', task='validity_classifier', device=0)

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taskb_training_df = pd.read_csv('../data/TaskB_train.csv')

taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')
taskb_valid_df = pd.read_csv('../data/TaskB_dev.csv')

In [6]:
taska_training_df = taska_training_df[taska_training_df.Validity != 0]
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '[CLS] {} [SEP] {} [SEP]'.format(x['Premise'], x['Conclusion']), axis=1)

In [7]:
resutls = arg_stance_pipeline(taska_training_df['input_txt'].tolist())
taska_training_df['pred_label'] = [1 if x['label'] == 'ENTAILMENT' else -1 for x in resutls]

In [8]:
taska_training_df[['topic', 'Validity', 'pred_label']].head(n=10)

,topic,Validity,pred_label
0,TV viewing is harmful to children,1,-1
1,TV viewing is harmful to children,1,-1
2,TV viewing is harmful to children,1,-1
3,TV viewing is harmful to children,1,1
4,TV viewing is harmful to children,-1,-1
5,TV viewing is harmful to children,1,1
6,TV viewing is harmful to children,1,1
7,TV viewing is harmful to children,1,1
8,TV viewing is harmful to children,-1,1
9,TV viewing is harmful to children,-1,-1


In [9]:
taska_training_df.Validity.value_counts()

 1    401
-1    320
Name: Validity, dtype: int64

In [10]:
precision, recall, f1, _ = precision_recall_fscore_support(taska_training_df.Validity.tolist(), taska_training_df.pred_label.tolist(), average='binary')

print('Precision: {}, Recall {}, F1: {}'.format(precision, recall, f1))

Precision: 0.8571428571428571, Recall 0.47880299251870323, F1: 0.6144


In [11]:
taska_valid_df = taska_valid_df[taska_valid_df.Validity != 0]
taska_valid_df['input_txt'] = taska_valid_df.apply(lambda x: '[CLS] {} [SEP] {} [SEP]'.format(x['Premise'], x['Conclusion']), axis=1)

## Fine-tune the NLI model on the training data:

In [12]:
nli_tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base-mnli')
nli_model     = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base-mnli')

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
train_dataset = Dataset.from_pandas(taska_training_df)
eval_dataset = Dataset.from_pandas(taska_valid_df)

In [14]:
nli_model.config.id2label

{0: 'CONTRADICTION', 1: 'NEUTRAL', 2: 'ENTAILMENT'}

In [15]:
validity_map = dict([
    (1, 2), # if valid -> entailment label
    (-1, 0) # not valid -> contradiction (neutral) label
])

In [16]:
inverse_validity_map = dict([
    (2,1),
    (1,-1),
    (0,-1) 
])

In [17]:
def preprocess(example):
    inputs = nli_tokenizer(example["input_txt"], add_special_tokens=False, padding=True, max_length=512)
    inputs['labels'] = list(map(validity_map.get, example['Validity']))
    return inputs

In [18]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
def compute_metrics(pred):
    labels = list(map(inverse_validity_map.get, pred.label_ids))
    preds = list(map(inverse_validity_map.get, pred.predictions.argmax(-1)))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'recall' : recall,
        'precision': precision,
        'f1': f1,
    }

In [20]:
training_args = TrainingArguments(
    output_dir="vali_nli_trainer", 
    report_to=None,
    num_train_epochs=5,
    learning_rate=2e-05
)

In [21]:
trainer = Trainer(
    model=nli_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: input_txt, Conclusion, Novelty, Premise, Novelty-Confidence, pred_label, Validity-Confidence, topic, Validity, __index_level_0__.
***** Running training *****
  Num examples = 721
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 455
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: majastahl. Use `wandb login --relogin` to force relogin


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=455, training_loss=0.2025895003434066, metrics={'train_runtime': 31.6894, 'train_samples_per_second': 113.761, 'train_steps_per_second': 14.358, 'total_flos': 252576689069250.0, 'train_loss': 0.2025895003434066, 'epoch': 5.0})

In [23]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: input_txt, Conclusion, Novelty, Premise, Novelty-Confidence, Validity-Confidence, topic, Validity, __index_level_0__.
***** Running Evaluation *****
  Num examples = 199
  Batch size = 8


{'eval_loss': 1.7431453466415405,
 'eval_recall': 0.936,
 'eval_precision': 0.7267080745341615,
 'eval_f1': 0.8181818181818182,
 'eval_runtime': 0.501,
 'eval_samples_per_second': 397.19,
 'eval_steps_per_second': 49.898,
 'epoch': 5.0}

In [24]:
preds = trainer.predict(eval_dataset)

The following columns in the test set  don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: input_txt, Conclusion, Novelty, Premise, Novelty-Confidence, Validity-Confidence, topic, Validity, __index_level_0__.
***** Running Prediction *****
  Num examples = 199
  Batch size = 8


## Fine-tune simple BERT model on the training data:

In [25]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model     = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//majaa/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/cep

In [26]:
train_dataset = Dataset.from_pandas(taska_training_df)
eval_dataset = Dataset.from_pandas(taska_valid_df)

In [27]:
validity_map = dict([ # avoid negative labels
    (1, 1), 
    (-1, 0)
])

In [28]:
def preprocess(example):
    inputs = bert_tokenizer(example["input_txt"], add_special_tokens=False, padding=True, truncation=True, max_length=512)
    inputs['labels'] = list(map(validity_map.get, example['Validity']))
    return inputs

In [29]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [30]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'recall' : recall,
        'precision': precision,
        'f1': f1,
    }

In [31]:
training_args = TrainingArguments(
    output_dir="vali_bert_trainer", 
    report_to="none",
    num_train_epochs=5,
    learning_rate=2e-05
)

PyTorch: setting up devices


In [32]:
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [36]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input_txt, Conclusion, Novelty, Premise, Novelty-Confidence, pred_label, Validity-Confidence, topic, Validity, __index_level_0__.
***** Running training *****
  Num examples = 721
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 455


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=455, training_loss=0.4069575718470982, metrics={'train_runtime': 19.9811, 'train_samples_per_second': 180.42, 'train_steps_per_second': 22.771, 'total_flos': 220455717176100.0, 'train_loss': 0.4069575718470982, 'epoch': 5.0})

In [37]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input_txt, Conclusion, Novelty, Premise, Novelty-Confidence, Validity-Confidence, topic, Validity, __index_level_0__.
***** Running Evaluation *****
  Num examples = 199
  Batch size = 8


{'eval_loss': 1.2082765102386475,
 'eval_recall': 0.944,
 'eval_precision': 0.6413043478260869,
 'eval_f1': 0.7637540453074434,
 'eval_runtime': 0.3022,
 'eval_samples_per_second': 658.612,
 'eval_steps_per_second': 82.74,
 'epoch': 5.0}

In [38]:
preds = trainer.predict(eval_dataset)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input_txt, Conclusion, Novelty, Premise, Novelty-Confidence, Validity-Confidence, topic, Validity, __index_level_0__.
***** Running Prediction *****
  Num examples = 199
  Batch size = 8
